### Modelo Regresivo Fama-French

In [1]:
import sklearn
import pandas as pd
import numpy as np

### Creando el conjunto de datos

Comentar las transformaciones del archivo original de Fama-French

In [237]:
df_ff = pd.read_csv('datos/F-F_Research_Data_Factors.CSV',index_col=False)
df_ff.drop(df_ff.columns[df_ff.columns.str.contains('Unnamed: 0')],axis=1,inplace=True)

In [238]:
df_ff

,Date,Mkt-RF,SMB,HML,RF
0,2000-01,-4.74,5.79,-1.91,0.41
1,2000-02,2.45,21.42,-9.70,0.43
2,2000-03,5.20,-17.23,8.17,0.47
3,2000-04,-6.40,-6.68,7.26,0.46
4,2000-05,-4.42,-6.09,4.81,0.50
...,...,...,...,...,...
266,2022-03,3.05,-1.60,-1.80,0.01
267,2022-04,-9.46,-1.41,6.19,0.01
268,2022-05,-0.34,-1.85,8.41,0.03
269,2022-06,-8.43,2.09,-5.97,0.06


In [239]:
df_ff['Date'] = pd.to_datetime(df_ff['Date'])

In [240]:
df_ff['Datetime'] = df_ff['Date']
df_ff.drop(['Date'],inplace=True,axis=1)
df_ff.set_index(['Datetime'],inplace=True)

In [241]:
df_ff_daily = df_ff.resample('b').ffill()
df_ff_daily = df_ff_daily.dropna()

In [242]:
df_ff_daily

,Mkt-RF,SMB,HML,RF
Datetime,,,,
2000-01-03,-4.74,5.79,-1.91,0.41
2000-01-04,-4.74,5.79,-1.91,0.41
2000-01-05,-4.74,5.79,-1.91,0.41
2000-01-06,-4.74,5.79,-1.91,0.41
2000-01-07,-4.74,5.79,-1.91,0.41
...,...,...,...,...
2022-06-27,-8.43,2.09,-5.97,0.06
2022-06-28,-8.43,2.09,-5.97,0.06
2022-06-29,-8.43,2.09,-5.97,0.06


### Uniendo datos de Fama-French a precios del S&P500

In [243]:
df_asset = pd.read_csv('datos/S&P500/AAPL.csv')

In [244]:
df_asset

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,retorno diario,retorno acumulado,banda_medio,banda_superior,banda_inferior,Conversion,Base,SpanA,SpanB,Lagging
0,2017-09-22,35.955339,36.128546,35.722819,36.038383,186581600,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.557053
1,2017-09-25,35.587579,36.024148,35.390647,35.720448,177549200,0.0,0.0,-0.008822,0.991178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.107502
2,2017-09-26,36.012282,36.520032,35.990929,36.334965,146640000,0.0,0.0,0.017204,1.008230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.597374
3,2017-09-27,36.491567,36.709852,36.429875,36.593590,102016800,0.0,0.0,0.007118,1.015406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.886848
4,2017-09-28,36.512923,36.605457,36.230576,36.368191,88022000,0.0,0.0,-0.006160,1.009152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.928436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2022-09-16,151.210007,151.350006,148.369995,150.699997,162157000,0.0,0.0,-0.010960,4.181653,159.837500,172.110437,147.564563,156.314995,162.259995,159.287495,149.100500,NaN
1255,2022-09-19,149.309998,154.559998,149.100006,154.479996,81474200,0.0,0.0,0.025083,4.286541,158.985500,170.205927,147.765072,156.314995,162.259995,159.287495,149.925504,NaN
1256,2022-09-20,153.399994,158.080002,153.080002,156.899994,107689800,0.0,0.0,0.015665,4.353691,158.451999,168.974657,147.929341,156.314995,162.259995,159.287495,150.515501,NaN
1257,2022-09-21,157.339996,158.740005,153.600006,153.720001,101407100,0.0,0.0,-0.020268,4.265452,157.776499,167.667363,147.885636,156.314995,162.259995,159.287495,151.125501,NaN


In [245]:
df_asset = df_asset[['Date','retorno diario']].copy()
df_asset.dropna(axis=0,inplace=True)
df_asset['Date'] = pd.to_datetime(df_asset['Date'])

In [246]:
df_asset

,Date,retorno diario
1,2017-09-25,-0.008822
2,2017-09-26,0.017204
3,2017-09-27,0.007118
4,2017-09-28,-0.006160
5,2017-09-29,0.005480
...,...,...
1254,2022-09-16,-0.010960
1255,2022-09-19,0.025083
1256,2022-09-20,0.015665
1257,2022-09-21,-0.020268


In [247]:
df_asset.set_index('Date',inplace=True)

In [248]:
df_asset

,retorno diario
Date,
2017-09-25,-0.008822
2017-09-26,0.017204
2017-09-27,0.007118
2017-09-28,-0.006160
2017-09-29,0.005480
...,...
2022-09-16,-0.010960
2022-09-19,0.025083
2022-09-20,0.015665


In [249]:
df_asset = df_asset.resample('m',label='right').sum()

In [250]:
df_asset

,retorno diario
Date,
2017-09-30,0.014820
2017-10-31,0.094045
2017-11-30,0.021484
2017-12-31,-0.014503
2018-01-31,-0.009661
...,...
2022-05-31,-0.046198
2022-06-30,-0.078820
2022-07-31,0.175835


In [251]:
start = df_asset.index.min()
end = df_asset.index.max()
start

Timestamp('2017-09-30 00:00:00', freq='M')

In [252]:
import pandas_datareader as reader
factors = reader.DataReader('F-F_Research_Data_Factors','famafrench',start,end)[0]
factors

,Mkt-RF,SMB,HML,RF
Date,,,,
2017-09,2.51,4.46,3.12,0.09
2017-10,2.25,-1.93,0.21,0.09
2017-11,3.12,-0.58,-0.08,0.08
2017-12,1.06,-1.32,0.05,0.09
2018-01,5.57,-3.15,-1.33,0.12
2018-02,-3.65,0.23,-1.07,0.11
2018-03,-2.35,4.05,-0.23,0.11
2018-04,0.28,1.14,0.54,0.14
2018-05,2.65,5.26,-3.18,0.14


In [253]:
factors.shape

(59, 4)

In [254]:
df_asset.shape

(61, 1)

In [255]:
df_asset = df_asset[:-2]
df_asset.shape

(59, 1)

In [256]:
df_asset.index = factors.index
df_asset

,retorno diario
Date,
2017-09,0.014820
2017-10,0.094045
2017-11,0.021484
2017-12,-0.014503
2018-01,-0.009661
2018-02,0.070756
2018-03,-0.056963
2018-04,-0.012498
2018-05,0.129059


In [257]:
df = pd.merge(df_asset,factors,on='Date')

In [258]:
df[['Mkt-RF','SMB','HML','RF']]=df[['Mkt-RF','SMB','HML','RF']]/100


In [259]:
df['ri-RF']=df['retorno diario']-df['RF']
df

,retorno diario,Mkt-RF,SMB,HML,RF,ri-RF
Date,,,,,,
2017-09,0.014820,0.0251,0.0446,0.0312,0.0009,0.013920
2017-10,0.094045,0.0225,-0.0193,0.0021,0.0009,0.093145
2017-11,0.021484,0.0312,-0.0058,-0.0008,0.0008,0.020684
2017-12,-0.014503,0.0106,-0.0132,0.0005,0.0009,-0.015403
2018-01,-0.009661,0.0557,-0.0315,-0.0133,0.0012,-0.010861
2018-02,0.070756,-0.0365,0.0023,-0.0107,0.0011,0.069656
2018-03,-0.056963,-0.0235,0.0405,-0.0023,0.0011,-0.058063
2018-04,-0.012498,0.0028,0.0114,0.0054,0.0014,-0.013898
2018-05,0.129059,0.0265,0.0526,-0.0318,0.0014,0.127659


In [260]:
Y = df['ri-RF']
X = df[['Mkt-RF','SMB','HML']]


### Inferencia con Statsmodel

In [265]:
import statsmodels.api as sm

X_sm = sm.add_constant(X)

model = sm.OLS(Y,X_sm)

result = model.fit()

In [266]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ri-RF   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     21.44
Date:                Fri, 23 Sep 2022   Prob (F-statistic):           2.52e-09
Time:                        12:42:48   Log-Likelihood:                 84.015
No. Observations:                  59   AIC:                            -160.0
Df Residuals:                      55   BIC:                            -151.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0159      0.008      1.977      0.053      -0.000       0.032
Mkt-RF         1.1809      0.158      7.456      0.000       0.864       1.498
SMB           -0.3384      0.297     -1.140      0.259      -0.933       0.257
HML           -0.5096      0.182     -2.799      0.007      -0.875      -0.145
==============================================================================
Omnibus:                       13.530   Durbin-Watson:                   2.200
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.103
Skew:                          -0.758   Prob(JB):                     2.62e-05
Kurtosis:                       5.507   Cond. No.                         38.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Predicciones con Sklearn

In [270]:
from sklearn.linear_model import SGDRegressor,LinearRegression

model = SGDRegressor()
model.fit(X,Y)

SGDRegressor()

In [271]:
model.score(X,Y)

-0.009292283924164035

In [272]:
model = LinearRegression()
model.fit(X,Y)

LinearRegression()

In [273]:
model.score(X,Y)

0.539048158438149

In [274]:
model.coef_

array([ 1.18091238, -0.33844107, -0.50960565])

In [275]:
model.intercept_

0.015885580908672654